<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8">
</head>
<body>
    
    
<div style="font-family:Georgia" align="center">
<h1 align="center">Quantum mechanics - Computation activities</h1>
<h1 align="center"><span style="text-decoration: gray underline overline" > Numerical integration of the Schrödinger equation</span></h1>
    
<br>
<br>
<br>
    
<h3> By: </h3>
<h3> Maxmime lepetit </h3>
<h3> Le trépané </h3>
    
<br>
<br>
<br>
    
<h3> Supervisor :</h3>
<h3> David Viennot </h3>
    
<br>
<br>
<br>

<h4> $5^{th}$ november 2022 <h4> 

    
    
</div>
</body> 
</html>

<h2 style="font-family:Georgia;text-decoration:gray underline">Introduction :</h2>

<h2 style="font-family:Georgia;text-decoration:gray underline">Theory :</h2>

<h2 style="font-family:Georgia;text-decoration:gray underline">Application:</h2>

<h2 style="font-family:Georgia;text-decoration:gray underline">Conclusion :</h2>